In [ ]:
import numpy as np

X = np.array([
    [25, 40000],
    [35, 60000],
    [45, 80000],
    [20, 20000],
    [35, 120000],
    [52, 18000],
    [23, 95000],
    [40, 62000],
    [60, 100000],
    [48, 220000],
    [33, 150000]
])
Y = np.array([0,0,0,0,0,0,1,1,1,1,1])

x_test = np.array([48, 142000])

classes = np.unique(Y)
stats = {}
for c in classes:
    Xc = X[Y == c]
    stats[c] = {
        'prior': Xc.shape[0] / X.shape[0],
        'mean': Xc.mean(axis=0),
        'std': Xc.std(axis=0, ddof=1) 
    }

def gaussian_pdf(x, mean, std):
    return (1 / (np.sqrt(2 * np.pi) * std)) * np.exp(-((x - mean) ** 2) / (2 * std ** 2))

posteriors = {}
for c in classes:
    prior = stats[c]['prior']
    mean = stats[c]['mean']
    std = stats[c]['std']
    likelihoods = gaussian_pdf(x_test, mean, std)
    likelihood = np.prod(likelihoods)
    posterior_unnorm = prior * likelihood
    posteriors[c] = posterior_unnorm
    print(f"Class {c}: prior={prior:.3f}, mean={mean}, std={std}, likelihoods={likelihoods}, unnorm posterior={posterior_unnorm:.6e}")

total = sum(posteriors.values())
post_norm = {c: posteriors[c]/total for c in classes}

print("\nNormalized posterior probabilities:", post_norm)
predicted = max(post_norm, key=post_norm.get)
print(f"\nPredicted class for sample {x_test.tolist()}: {predicted}")

Class 0: prior=0.545, mean=[3.53333333e+01 5.63333333e+04], std=[1.19443152e+01 3.92003401e+04], likelihoods=[1.90345547e-02 9.34487222e-07], unnorm posterior=9.702299e-09
Class 1: prior=0.455, mean=[4.080e+01 1.254e+05], std=[1.4131525e+01 6.1528855e+04], likelihoods=[2.47943036e-02 6.25209496e-06], unnorm posterior=7.046197e-08

Normalized posterior probabilities: {np.int64(0): np.float64(0.1210302140212407), np.int64(1): np.float64(0.8789697859787593)}

Predicted class for sample [48, 142000]: 1


In [ ]:
import numpy as np
import pandas as pd
from math import sqrt, pi, exp

df = pd.read_csv("diabetes.csv")  


X = df.iloc[:, :-1].values
y = df.iloc[:, -1].values

data = np.hstack((X, y.reshape(-1,1)))

def summarize_by_class(X_train, y_train):
    classes = np.unique(y_train)
    summaries = {}
    priors = {}
    total_samples = len(y_train)

    for c in classes:
        X_c = X_train[y_train == c]
        priors[c] = len(X_c) / total_samples
        means = X_c.mean(axis=0)
        variances = X_c.var(axis=0) + 1e-9   
        summaries[c] = (means, variances)

    return summaries, priors

def gaussian_pdf(x, mean, var):
    return (1 / sqrt(2 * pi * var)) * exp(-((x - mean)**2) / (2 * var))

def predict_sample(x, summaries, priors):
    posteriors = {}

    for c, (means, variances) in summaries.items():
        prior = priors[c]
        likelihood = 1
        for i in range(len(x)):
            likelihood *= gaussian_pdf(x[i], means[i], variances[i])
        posteriors[c] = prior * likelihood

    return max(posteriors, key=posteriors.get)

np.random.shuffle(data)
fold_size = len(data) // 2

fold1 = data[:fold_size]
fold2 = data[fold_size:]

def evaluate(train, test):
    X_train, y_train = train[:, :-1], train[:, -1]
    X_test, y_test = test[:, :-1], test[:, -1]

    summaries, priors = summarize_by_class(X_train, y_train)

    predictions = []
    for sample in X_test:
        predictions.append(predict_sample(sample, summaries, priors))

    accuracy = np.mean(predictions == y_test) * 100
    return accuracy

acc1 = evaluate(fold1, fold2)
acc2 = evaluate(fold2, fold1)

print("Accuracy Fold-1:", acc1, "%")
print("Accuracy Fold-2:", acc2, "%")
print("Average Accuracy:", (acc1 + acc2)/2, "%")


Accuracy Fold-1: 76.30208333333334 %
Accuracy Fold-2: 73.4375 %
Average Accuracy: 74.86979166666667 %


Naïve Bayes assumes conditional independence between features and therefore uses only feature-wise likelihoods. It is simple, fast, and works well when features are independent or the dataset is small.

The Multivariate Bayesian Classification Model (e.g., QDA, LDA) uses a full covariance matrix to model feature interactions and correlations. It is more powerful but requires more data and computation.